https://www.unisender.com/ru/glossary/mau-dau-ili-osnovnye-metriki-mobilnyh-prilozhenij/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)

In [9]:
df = pd.read_excel('dataset-1.xlsx')
df.head()

,Дата,Session ID,Client ID,Ярлык события,Доход
0,20180902.0,6517411217615201018blqo010gfv,42325189.0,Search Field,0.0
1,20180902.0,6517439917954201018jogp8guf,49083805.0,Movie cover,0.0
2,20180903.0,6517492978275201018xnvoyeqm,27598678.0,Event Place,0.0
3,20180903.0,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746.0,Movie cover,0.0
4,20180903.0,6517492978275201018xnvoyeqm,27598678.0,Main - Menu,0.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20342 entries, 0 to 20341
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Дата           20342 non-null  datetime64[ns]
 1   Session ID     20342 non-null  object        
 2   Client ID      20342 non-null  float64       
 3   Ярлык события  20342 non-null  object        
 4   Доход          20342 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 794.7+ KB


In [38]:
df['Дата'] = pd.to_datetime(df['Дата'], format='%Y%m%d')
df['Месяц'] = df['Дата'].dt.month
df['День'] = df['Дата'].dt.day

## Количество сессий (визитов).

In [133]:
n_session = df['Session ID'].nunique()
n_session

16907

## MAU (Monthly Active Users)

Это количество уникальных пользователей за месяц

In [93]:
mau = df.groupby('Месяц')['Client ID'].agg(mau='nunique')
mau

,mau
Месяц,
9,2511
10,8520
11,3647


## DAU (Daily Active Users) 

Это количество уникальных пользователей за сутки.

In [39]:
df.groupby(['Месяц', 'День'])['Client ID'].agg(['nunique'])

nunique
Месяц День         
9     2           2
      3           2
      5           1
      6           2
      7           1
      8           2
      9           1
      10          2
      11          2
      12         32
      13        110
      14        160
      15        137
      16         79
      17        240
      18         97
      19        107
      20        137
      21        239
      22        145
      23         98
      24        228
      25         83
      26        116
      27        158
      28        215
      29        175
      30        105
10    1         228
      2         147
      3         264
      4         385
      5         552
      6         419
      7         222
      8         304
      9         192
      10        236
      11        363
      12        504
      13        331
      14        165
      15        330
      16        173
      17        225
      18        323
      19        522
      20        382
      21        145
      22        297
      23        124
      24        186
      25        279
      26        454
      27        380
      28        215
      29        458
      30        233
      31        299
11    1         309
      2         511
      3         598
      4         410
      5         276
      6         122
      7         130
      8         198
      9         377
      10        282
      11        129
      12        274
      13        111
      14         83
      15          7
      17          3
      18          2
      19          2
      20          2
      21          2
      22          1
      23          1
      24          1
      25          3
      26          3
      27          2
      29          1
      30          4

## Revenue

In [122]:
revenue = df.groupby('Месяц')['Доход'].agg(revenue='sum')
revenue

,revenue
Месяц,
9,2.779013e+07
10,2.986842e+07
11,1.960508e+07


## ARPU (Average Revenue Per User) 

Cредняя прибыль с одного клиента за конкретный период

In [47]:
arpu = revenue.values / mau.values
arpu

array([[11067.35386891],
       [ 3505.68298277],
       [ 5375.67439391]])

## Paying Users

Число платящих пользователей

In [115]:
revenue_client = df.groupby(['Месяц', 'Client ID'])['Доход'].agg(['sum'])
revenue_client = revenue_client[revenue_client['sum'] > 0]
revenue_client

sum
Месяц Client ID               
9     28662.0      3735.648491
      36906.0     55342.940600
      197702.0    17266.997467
      221917.0     4980.864654
      242234.0    16602.882180
...                        ...
11    78513984.0   4427.435248
      78616572.0  11068.588120
      78676662.0  24904.323270
      78858836.0   3735.648491
      79042557.0  29885.187924

[3353 rows x 1 columns]

In [116]:
pu = revenue_client.groupby('Месяц')['sum'].agg(paying_users='count')
pu

,paying_users
Месяц,
9,1268
10,1195
11,890


## ARPPU (Average Revenue Per Paying User) 

Cредняя прибыль с одного платящего клиента за конкретный период

In [124]:
arppu = revenue.values / pu.values
arppu

array([[21916.50281139],
       [24994.49289805],
       [22028.18484784]])

## Количество сессий на пользователя 

Session / user

In [134]:
n_session / mau.values

array([[6.73317403],
       [1.98438967],
       [4.63586509]])

## Количество целевых действий

(целевое действие — событие, за которое получен доход)

In [135]:
events = df[df['Доход'] > 0]
events = events.groupby('Ярлык события')['Ярлык события'].agg(n_events='count')
events

,n_events
Ярлык события,
Desktop - To Payment Card,1195
Mobile - To Payment Card,1652
Movie cover,2
Place Select,1277
Promocode,144
Search Field,1


## LTV (Lifetime Value), он же CLV (Customer Lifetime Value) 

Cреднее количество денег от одного пользователя за всё время использования приложения.

In [129]:
arpu.sum()

19948.71124558516

---
# Практика 2

In [7]:
df = pd.read_excel('Dataset-2.xlsx', sheet_name='Лист2')
df.head()

,Date,Выручка,% доступности,MAU,Новые,Вернувшиеся,Количество операций,Ошибки,Средний чек,% доступности.1,MAU.1,Новые.1,Вернувшиеся.1,Количество операций.1,Ошибки.1,Средний чек.1,Доля платящих
0,02.сент,600000.0,91,6209971.0,1271736,4938235.0,1218.323587,3715,492.480000,0.703224,0.884078,-0.069135,0.854884,0.342768,-0.34811,0.656413,NaN
1,03.сент,539800.0,98,6138791.0,1447192,4691599.0,1747.030049,3200,308.981520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,05.сент,809765.0,92,6392179.0,1195080,5197099.0,2017.756255,2915,401.319534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06.сент,978651.0,100,7697321.0,1387697,6309624.0,2240.143369,1550,436.869806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07.сент,769542.0,97,6698070.0,1752918,4945152.0,1578.282828,1501,487.581811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df = df.iloc[:, 1:9]
df.head()

,Выручка,% доступности,MAU,Новые,Вернувшиеся,Количество операций,Ошибки,Средний чек
0,600000.0,91,6209971.0,1271736,4938235.0,1218.323587,3715,492.480000
1,539800.0,98,6138791.0,1447192,4691599.0,1747.030049,3200,308.981520
2,809765.0,92,6392179.0,1195080,5197099.0,2017.756255,2915,401.319534
3,978651.0,100,7697321.0,1387697,6309624.0,2240.143369,1550,436.869806
4,769542.0,97,6698070.0,1752918,4945152.0,1578.282828,1501,487.581811


In [22]:
df.corr()

,Выручка,% доступности,MAU,Новые,Вернувшиеся,Количество операций,Ошибки,Средний чек
Выручка,1.000000,0.703224,0.884078,-0.069135,0.854884,0.342768,-0.348110,0.656413
% доступности,0.703224,1.000000,0.573610,-0.025328,0.550217,0.553472,-0.505607,0.324111
MAU,0.884078,0.573610,1.000000,-0.108958,0.973990,0.255222,-0.231247,0.633347
Новые,-0.069135,-0.025328,-0.108958,1.000000,-0.331367,-0.102234,0.120925,0.054064
Вернувшиеся,0.854884,0.550217,0.973990,-0.331367,1.000000,0.265549,-0.247054,0.588819
Количество операций,0.342768,0.553472,0.255222,-0.102234,0.265549,1.000000,-0.661127,-0.386178
Ошибки,-0.348110,-0.505607,-0.231247,0.120925,-0.247054,-0.661127,1.000000,0.223348
Средний чек,0.656413,0.324111,0.633347,0.054064,0.588819,-0.386178,0.223348,1.000000


In [24]:
df_corr = df.corr().iloc[0]
df_corr[df_corr.abs() > 0.7]

Выручка          1.000000
% доступности    0.703224
MAU              0.884078
Вернувшиеся      0.854884
Name: Выручка, dtype: float64